In [3]:
from fastai.vision.all import *
import torchvision 
import numpy as np
import torch
import cv2
import os
from modules_th.video_block import *
import random
import time

In [4]:
mean = [0.43216, 0.394666, 0.37645]
std = [0.22803, 0.22145, 0.216989]

model = create_body(torchvision.models.video.r2plus1d_18, cut=-2)

In [8]:
vid_paths = get_files('/mnt/data/eugeniomarinelli/UCF_Crimes/Videos', extensions='.mp4')
vid_path = random.choice(vid_paths)
for frame in create_video_generator(vid_path):
    print(frame.__array_interface__['data'][0])

94865077762896
94865077762896
94865077762896
94865077762896
94865077762896
94865077762896
94865077762896
94865077762896
94865077762896
94865077762896
94865077762896
94865077762896
94865077762896
94865077762896
94865077762896
94865077762896
94865077762896
94865077762896
94865077762896
94865077762896
94865077762896
94865077762896
94865077762896
94865077762896
94865077762896
94865077762896
94865077762896
94865077762896
94865077762896
94865077762896
94865077762896
94865077762896
94865077762896
94865077762896
94865077762896
94865077762896
94865077762896
94865077762896
94865077762896
94865077762896
94865077762896
94865077762896
94865077762896
94865077762896
94865077762896
94865077762896
94865077762896
94865077762896
94865077762896
94865077762896
94865077762896
94865077762896
94865077762896
94865077762896
94865077762896
94865077762896
94865077762896
94865077762896
94865077762896
94865077762896
94865077762896
94865077762896
94865077762896
94865077762896
94865077762896
94865077762896
9486507776

KeyboardInterrupt: 

In [11]:
def initialize_tensors(bs, l, size):
    fr_ts = torch.zeros(3, size, size, dtype=float)
    snip = torch.zeros(l, 3, size, size, dtype=float)
    b = torch.zeros(bs, l, 3, size, size, dtype=float)
    return fr_ts, snip, b

def iterate_with_vidcap(vid_paths):
    for vid_path in vid_paths:
        vidcap = cv2.VideoCapture(str(vid_path))
        yield vidcap, vid_path
        vidcap.release()

def iterator_over_frames(vidcap, skip=2):
    vidcap = cv2.VideoCapture(str(vid_path))
    duration = vidcap.get(cv2.CAP_PROP_FRAME_COUNT)
    height = int(vidcap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    width =  int(vidcap.get(cv2.CAP_PROP_FRAME_WIDTH))
    l = int(duration//skip)
    frame = np.zeros((height,width,3), dtype=np.uint8)
    for frame_pos in range(0, l*skip, skip):
        vidcap.set(cv2.CAP_PROP_POS_FRAMES, frame_pos)
        res, frame[:] = vidcap.read()
        yield frame        
        
def snippet_is_ready(i, l): return i==l-1
def batch_is_ready(i, l, j, bs): j==bs-1
        
def get_and_save_embs(b, model, path):
    embs = model(b)
    for emb in embs:
         torch.save(emb, path) ### REMEMBER TO PUT PATH!!

def create_embeddings(vid_paths, model, pl, bs, size, l, skip):
    fr_ts, snip, b = initialize_tensors(bs, l, size)
    for vidcap, vid_path in iterate_with_vidcap(vid_paths):        
        j = 0 # j contera' quanti snippets sono stati gia' messi nel batch       
        for n_fr, frame in iterator_over_frames(vidcap, skip=skip):
            i, n_sn = n_fr%l,  n_fr//l  # i contera' il # di frames messo nello snippet, n_sn e' il numero di snips creati finora             
            fr_ts[:] = pl(frame)
            snip[i] = fr_ts
            if snippet_is_ready(i,l): b[j] = snip; j += 1 
            if batch_is_ready(j,bs): get_and_save_embs(b,path); j = 0
        if not batch_is_ready(j,bs): get_and_save_embs(b, path)  # if the video didnt fill the last batch, save nevertheless the 
                                                                 # last created snipets

In [ ]:
files = get_files('/mnt/data/eugeniomarinelli/UCF_Crimes/Videos',extensions='.pt')
files.map(os.remove)

(#76600) [None,None,None,None,None,None,None,None,None,None...]